# Descriptive Analysis
In this section we look at the data from a descriptive perspective. This gives a good understanding and overview on the test data. 

In [27]:
import pandas as pd
from currency_converter import CurrencyConverter
import datetime as dt
from datetime import datetime, timedelta
current_datetime = "{:%Y_%m_%d_%H-%M-%S}".format(datetime.now())

In [28]:
data = pd.read_csv('./data/test.csv',parse_dates=['Flight_Date','Request_Date'])
train = pd.read_csv('./data/train.csv',parse_dates=['Flight_Date','Request_Date'])
data

,Request_Date,Flight_Date,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency
0,2019-06-03 11:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,26.87,£
1,2019-06-03 23:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£
2,2019-06-04 11:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£
3,2019-06-04 23:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£
4,2019-06-05 11:00:00+00:00,2019-07-02,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£
...,...,...,...,...,...,...,...,...
5578,2019-07-30 23:00:00+00:00,2019-08-02,22,2019-08-02 FR 8545,SXF-STN,FR 8545,42.83,€
5579,2019-07-31 11:00:00+00:00,2019-08-02,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€
5580,2019-07-31 23:00:00+00:00,2019-08-02,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€
5581,2019-08-01 11:00:00+00:00,2019-08-02,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€


In [29]:
# Compute pound values to eur (currency_in_eur)
c = CurrencyConverter(fallback_on_missing_rate=True)
def convert_to_eur(row, column):
    if row['Currency'] == '£':
        dateString = datetime.strftime(row['Request_Date'], '%Y-%m-%d')
        date = dt.datetime.strptime(dateString, "%Y-%m-%d")
        return c.convert(row[column], 'GBP', 'EUR', date=datetime(date.year, date.month, date.day))
    return row[column]

data['price_in_eur'] = data.apply(lambda row : convert_to_eur(row, 'Price'), axis=1)
train['price_in_eur'] = train.apply(lambda row : convert_to_eur(row, 'Price'), axis=1)
train['min_future_price_in_eur'] = train.apply(lambda row : convert_to_eur(row, 'min_future_price'), axis=1)

In [30]:
def modify_date_columns(data):
    # Formatting necessary to do duration calculation
    data['Departure_hour'] = pd.to_numeric(data['Departure_hour'])
    data['Request_Date'] = data['Request_Date'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H:%M:%S'))
    data['Flight_Date'] = data['Flight_Date'] + pd.to_timedelta(data['Departure_hour'], 'h')
    data['Request_Date'] = pd.to_datetime(data['Request_Date'])
    # Calculate duration between flight_date and request_date in hours
    Duration_Time = data['Flight_Date'] - data['Request_Date']
    Duration_Time = Duration_Time.apply(lambda x: x.total_seconds())     
    Duration_Time = divmod(Duration_Time, 3600)[0]
    data.insert(2, "Duration_Time", Duration_Time, True)
    return data

data = modify_date_columns(data)
train = modify_date_columns(train)

In [31]:
# New feature: Compute weekday from Request_date
data['Request_Weekday'] = data['Request_Date'].dt.day_name()
train['Request_Weekday'] = train['Request_Date'].dt.day_name()

In [32]:
# New feature: Compute day, noon, night from Request_date
def get_departureReq(req):
    req = req.hour
    if (req >= 6 and req < 12):
        return 'Morning'
    elif (req >= 12 and req < 17):
        return 'Noon'
    elif (req >= 17 and req < 20):
        return 'Evening'
    else:
        return 'Night'

data['Request_timeofday'] = data['Request_Date'].apply(get_departureReq)
train['Request_timeofday'] = train['Request_Date'].apply(get_departureReq)

In [33]:
# New feature: Compute day, noon, night from Departure_Date
def get_departureDep(dep):
    if (dep >= 6 and dep < 12):
        return 'Morning'
    elif (dep >= 12 and dep < 17):
        return 'Noon'
    elif (dep >= 17 and dep < 20):
        return 'Evening'
    else:
        return 'Night'

data['Departure_timeofday'] = data['Departure_hour'].apply(get_departureDep)
train['Departure_timeofday'] = train['Departure_hour'].apply(get_departureDep)
data

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday
0,2019-06-03 11:00:00,2019-07-02 07:00:00,692.0,7,2019-07-02 FR 145,STN-SXF,FR 145,26.87,£,23.811657,Monday,Morning,Morning
1,2019-06-03 23:00:00,2019-07-02 07:00:00,680.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.968663,Monday,Night,Morning
2,2019-06-04 11:00:00,2019-07-02 07:00:00,668.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Morning,Morning
3,2019-06-04 23:00:00,2019-07-02 07:00:00,656.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Night,Morning
4,2019-06-05 11:00:00,2019-07-02 07:00:00,644.0,7,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.973793,Wednesday,Morning,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5578,2019-07-30 23:00:00,2019-08-02 22:00:00,71.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,42.83,€,42.830000,Tuesday,Night,Night
5579,2019-07-31 11:00:00,2019-08-02 22:00:00,59.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Morning,Night
5580,2019-07-31 23:00:00,2019-08-02 22:00:00,47.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Night,Night
5581,2019-08-01 11:00:00,2019-08-02 22:00:00,35.0,22,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Thursday,Morning,Night


In [34]:
# New feature: Distance travelled
# https://www.aircalculator.com/flightplan.php?from=SXF&to=STN
def getDistance(route):
    if (route == 'STN-SXF' or route == 'SXF-STN'):
        return 907.20
    elif (route == 'FRA-STN' or route == 'STN-FRA'):
        return 618.63

data['flight_distance_km'] = data['route_abb'].apply(getDistance)
train['flight_distance_km'] = train['route_abb'].apply(getDistance)

In [35]:
# New feature: Request hour
data['Request_hour'] = data['Request_Date'].apply(lambda x: x.strftime('%H:%M'))
train['Request_hour'] = train['Request_Date'].apply(lambda x: x.strftime('%H:%M'))
train['Departure_hour'] = train['Departure_hour'].apply(lambda x: str(x) + ':00')
data['Departure_hour'] = data['Departure_hour'].apply(lambda x: str(x) + ':00')

In [36]:
# Remove hour from date in Request and Flight date
data['Request_Date_Month'] = data['Request_Date'].apply(lambda x: int(x.strftime('%m')))
data['Flight_Date_Month'] = data['Flight_Date'].apply(lambda x: int(x.strftime('%m')))
train['Request_Date_Month'] = train['Request_Date'].apply(lambda x: int(x.strftime('%m')))
train['Flight_Date_Month'] = train['Flight_Date'].apply(lambda x: int(x.strftime('%m')))

In [37]:
# Split routes
def clean_route_from(route):
    route = str(route)
    route = route.split('-')
    return route[0]

def clean_route_to(route):
    route = str(route)
    route = route.split('-')
    return route[1]

data['route_abb_from'] = data['route_abb'].apply(clean_route_from)
data['route_abb_to'] = data['route_abb'].apply(clean_route_to)
train['route_abb_from'] = train['route_abb'].apply(clean_route_from)
train['route_abb_to'] = train['route_abb'].apply(clean_route_to)

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday,flight_distance_km,Request_hour,Request_Date_Month,Flight_Date_Month,route_abb_from,route_abb_to
0,2019-06-03 11:00:00,2019-07-02 07:00:00,692.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,26.87,£,23.811657,Monday,Morning,Morning,907.2,11:00,6,7,STN,SXF
1,2019-06-03 23:00:00,2019-07-02 07:00:00,680.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.968663,Monday,Night,Morning,907.2,23:00,6,7,STN,SXF
2,2019-06-04 11:00:00,2019-07-02 07:00:00,668.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Morning,Morning,907.2,11:00,6,7,STN,SXF
3,2019-06-04 23:00:00,2019-07-02 07:00:00,656.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Night,Morning,907.2,23:00,6,7,STN,SXF
4,2019-06-05 11:00:00,2019-07-02 07:00:00,644.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.973793,Wednesday,Morning,Morning,907.2,11:00,6,7,STN,SXF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5578,2019-07-30 23:00:00,2019-08-02 22:00:00,71.0,22:00,2019-08-02 FR 8545,SXF-STN,FR 8545,42.83,€,42.830000,Tuesday,Night,Night,907.2,23:00,7,8,SXF,STN
5579,2019-07-31 11:00:00,2019-08-02 22:00:00,59.0,22:00,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Morning,Night,907.2,11:00,7,8,SXF,STN
5580,2019-07-31 23:00:00,2019-08-02 22:00:00,47.0,22:00,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Night,Night,907.2,23:00,7,8,SXF,STN
5581,2019-08-01 11:00:00,2019-08-02 22:00:00,35.0,22:00,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Thursday,Morning,Night,907.2,11:00,8,8,SXF,STN


In [38]:
data

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday,flight_distance_km,Request_hour,Request_Date_Month,Flight_Date_Month,route_abb_from,route_abb_to
0,2019-06-03 11:00:00,2019-07-02 07:00:00,692.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,26.87,£,23.811657,Monday,Morning,Morning,907.2,11:00,6,7,STN,SXF
1,2019-06-03 23:00:00,2019-07-02 07:00:00,680.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.968663,Monday,Night,Morning,907.2,23:00,6,7,STN,SXF
2,2019-06-04 11:00:00,2019-07-02 07:00:00,668.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Morning,Morning,907.2,11:00,6,7,STN,SXF
3,2019-06-04 23:00:00,2019-07-02 07:00:00,656.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,35.016015,Tuesday,Night,Morning,907.2,23:00,6,7,STN,SXF
4,2019-06-05 11:00:00,2019-07-02 07:00:00,644.0,7:00,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,34.973793,Wednesday,Morning,Morning,907.2,11:00,6,7,STN,SXF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5578,2019-07-30 23:00:00,2019-08-02 22:00:00,71.0,22:00,2019-08-02 FR 8545,SXF-STN,FR 8545,42.83,€,42.830000,Tuesday,Night,Night,907.2,23:00,7,8,SXF,STN
5579,2019-07-31 11:00:00,2019-08-02 22:00:00,59.0,22:00,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Morning,Night,907.2,11:00,7,8,SXF,STN
5580,2019-07-31 23:00:00,2019-08-02 22:00:00,47.0,22:00,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Wednesday,Night,Night,907.2,23:00,7,8,SXF,STN
5581,2019-08-01 11:00:00,2019-08-02 22:00:00,35.0,22:00,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,50.990000,Thursday,Morning,Night,907.2,11:00,8,8,SXF,STN


In [39]:
data.to_csv('./data/converted_test_' + current_datetime + '.csv')

In [40]:
train.to_csv('./data/converted_train_' + current_datetime + '.csv')